In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq


data_files = {
    'train': '../data/DuReaderQG/train.json',
    'validation': '../data/DuReaderQG/dev.json'
}
raw_datasets = load_dataset("json", data_files=data_files)
print(raw_datasets)
print(raw_datasets['train'].features)

/Users/zhouke/Documents/project/stage3/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['context', 'answer', 'question', 'id'],
        num_rows: 14520
    })
    validation: Dataset({
        features: ['context', 'answer', 'question', 'id'],
        num_rows: 984
    })
})
{'context': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'id': Value(dtype='int64', id=None)}


In [2]:
from transformers import AutoModelForSeq2SeqLM

checkpoint = 'langboat/mengzi-t5-base'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
import torch
def tokenize_function(example):
    result = tokenizer(
        example['question'],
        example['context'],
        truncation=True,
        padding=True,
        return_tensors='pt'
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example['answer'],
            truncation=True,
            padding=True,
            return_tensors='pt'
        )['input_ids']
    #     result['decoder_input_ids'] = model.prepare_decoder_input_ids_from_labels(labels)
    #     end_token_idx = torch.where(labels == tokenizer.eos_token_id)[1]
    #     for idx, end_idx in enumerate(end_token_idx):
    #         labels[idx][end_idx + 1:] = -100
        example['labels'] = labels
    # example['labels'] = example['answer']
    return result

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/14520 [00:00<?, ? examples/s]/Users/zhouke/Documents/project/stage3/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 14520/14520 [00:03<00:00, 4779.83 examples/s]


In [11]:
tokenizer(raw_datasets['train'][0]['context'])['attention_mask']

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model, padding=True)

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model=model, peft_config=peft_config)
model.print_trainable_parameters()

In [ ]:
tokenized_datasets.remove_columns(['context', 'question', 'answer'])

In [ ]:
from transformers import TrainingArguments, Seq2SeqTrainingArguments

# training_args = TrainingArguments('output/model', per_device_train_batch_size=4, per_device_eval_batch_size=4, eval_strategy='epoch')
training_args = TrainingArguments(
    'output/model',
    per_device_train_batch_size=4,   # 调小 batch size
    per_device_eval_batch_size=4,    # 调小 eval batch size
    eval_strategy='epoch',
    fp16=True,
    
)
# training_args = TrainingArguments('output/model', eval_strategy='epoch')
# training_args = Seq2SeqTrainingArguments(
#     'output/model',
#     per_device_train_batch_size=4,   # 调小 batch size
#     per_device_eval_batch_size=4,    # 调小 eval batch size
#     eval_strategy='epoch',
#     label_names=['labels']
# )

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [ ]:
trainer.train()

In [1]:
model.generate(tokenized_datasets['dev'][0])

NameError: name 'model' is not defined

In [3]:
true_predictions = ['I-中医证候', 'O', 'I-中医诊断', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'B-中医证候', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现'], ['I-中医证候', 'B-西医诊断', 'I-中医证候', 'B-临床表现', 'I-中医诊断', 'B-西医诊断', 'I-中医治疗', 'I-中医证候', 'I-临床表现', 'I-中医证候', 'O', 'B-临床表现', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'B-临床表现', 'O', 'B-临床表现', 'I-中医证候'], ['O', 'B-临床表现', 'I-方剂', 'I-中医证候', 'I-方剂', 'B-临床表现', 'I-中医证候', 'I-方剂', 'I-中药', 'I-方剂', 'O', 'B-西医诊断', 'I-方剂', 'I-方剂', 'B-中医证候', 'B-临床表现', 'B-西医诊断', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-方剂', 'I-中医证候', 'I-中医证候', 'B-中医诊断', 'B-西医诊断', 'O', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'O'], ['I-中医证候', 'B-西医诊断', 'B-西医诊断', 'B-中医证候', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'B-临床表现', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'I-中医证候'], ['I-中医证候', 'B-西医诊断', 'B-西医诊断', 'O', 'I-中医证候', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'O', 'O', 'I-西医诊断', 'I-西医诊断', 'I-中医证候', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'B-西医诊断', 'B-临床表现', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'I-西医诊断', 'I-中医证候', 'I-中医证候', 'B-临床表现', 'B-西医诊断', 'I-中医证候', 'O', 'B-临床表现', 'B-西医诊断', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'O', 'I-中医证候', 'B-西医诊断', 'I-中医证候', 'I-中医证候', 'I-中医证候', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-临床表现', 'I-中医证候', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'I-中医证候', 'B-临床表现', 'I-中医证候', 'I-中医证候', 'I-西医诊断', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'O', 'B-西医诊断', 'B-临床表现', 'I-方剂', 'I-中医治疗', 'I-方剂', 'B-临床表现', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-西医诊断', 'B-临床表现', 'B-西医诊断', 'B-临床表现', 'B-临床表现', 'I-中医证候', 'I-西医诊断', 'B-西医诊断', 'I-中医证候']
true_labels = ['O', 'O', 'O', 'O', 'O', 'O', 'B-中医诊断', 'I-中医诊断', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

len(true_labels) == len(true_predictions)


False